### 2.4.3 データ主導プログラミングと加法性


In [1]:
(define (lookup key table)
  (let ((record (assoc key (cdr table))))
    (if record (cdr record)
        #f)
    )
  )

(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (insert! key value table)
  (let ((record (assoc key (cdr table))))
    (if record (set-cdr! record value)
        (set-cdr! table (cons (cons key value) (cdr table)))
        )
    )
  'ok)

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

;(define (make-table) (list '*table*))

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))

In [2]:
; put手続きの動作確認
(put 'key1 '(key2) "test")

ok

In [3]:
; get手続きの動作
(get 'key1 '(key2))

"test"

In [7]:
(import "math")
(define (make-from-mag-ang r a)
  (define (dispatch op)
    (cond ((eq? op 'real-part) (* r (math.cos a)))
          ((eq? op 'imag-part) (* r (math.sin a)))
          ((eq? op 'magnitude) r)
          ((eq? op 'angle) a)
          (else (error "Unknown op: MAKE-FROM-MAG-ANG" op))
          )
  )
dispatch)

(define (square x)(* x x))

(define (install-rectangular-package)
;; 内部⼿続き
(define (real-part z) (car z))
(define (imag-part z) (cdr z))
(define (make-from-real-imag x y) (cons x y))
(define (magnitude z)
(sqrt (+ (square (real-part z))
(square (imag-part z)))))
(define (angle z)
(atan (imag-part z) (real-part z)))
(define (make-from-mag-ang r a)
(cons (* r (cos a)) (* r (sin a))))
;; システムのほかの部分とのインターフェイス
(define (tag x) (attach-tag 'rectangular x))
(put 'real-part '(rectangular) real-part)
(put 'imag-part '(rectangular) imag-part)
(put 'magnitude '(rectangular) magnitude)
(put 'angle '(rectangular) angle)
(put 'make-from-real-imag 'rectangular
(lambda (x y) (tag (make-from-real-imag x y))))
(put 'make-from-mag-ang 'rectangular
(lambda (r a) (tag (make-from-mag-ang r a))))
'done)

(define (install-polar-package)
;; 内部⼿続き
(define (magnitude z) (car z))
(define (angle z) (cdr z))
(define (make-from-mag-ang r a) (cons r a))
(define (real-part z) (* (magnitude z) (cos (angle z))))
(define (imag-part z) (* (magnitude z) (sin (angle z))))
(define (make-from-real-imag x y)
(cons (sqrt (+ (square x) (square y)))
(atan y x)))
;; システムのほかの部分とのインターフェイス
(define (tag x) (attach-tag 'polar x))
(put 'real-part '(polar) real-part)
(put 'imag-part '(polar) imag-part)
(put 'magnitude '(polar) magnitude)
(put 'angle '(polar) angle)
(put 'make-from-real-imag 'polar
(lambda (x y) (tag (make-from-real-imag x y))))
(put 'make-from-mag-ang 'polar
(lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (make-from-real-imag x y)
((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a)
((get 'make-from-mag-ang 'polar) r a))

;(install-rectangular-package)
(install-polar-package)


Traceback (most recent call last):
  File "In [7]", line 58, col 1, in 'install-polar-package'
  Source "macro-generated-exp"
  File "In [7]", line 37, col 2
RunTimeError: unbound variable 'put'



#### メッセージパッシング


In [27]:
(import "math")
(define (make-from-real-imag x y)
  (define (dispatch op)
    (cond ((eq? op 'real-part) x)
          ((eq? op 'imag-part) y)
          ((eq? op 'magnitude) (sqrt (+ (square x) (square y))))
          ;((eq? op 'angle) (atan y x))
          ((eq? op 'angle) (math.atan2 y x))
          (else (error "Unknown op: MAKE-FROM-REAL-IMAG" op))))
dispatch)

(define (square x)(* x x))
(define (apply-generic op arg) (arg op))
(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))


In [28]:
(define z (make-from-real-imag 1 2))

In [29]:
(real-part z)

1

In [30]:
(imag-part z)

2

In [31]:
(magnitude z)

2.23606797749979

In [32]:
(angle z)

1.1071487177940904



#### 練習問題

- [練習問題2.73 記号微分のデータ主導システム](../exercises/2.73.ipynb)
- [練習問題2.74 貪欲エンタープライズ社](../exercises/2.74.ipynb)
- [練習問題2.75 make-from-mag-ang](../exercises/2.75.ipynb)
- [練習問題2.76 三つの戦略](../exercises/2.76.ipynb)